In [ ]:
!pip install kaggle
mkdir ~/.kaggle
mv kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json



In [ ]:
# Step 1: Data Extraction and Preprocessing

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



In [ ]:
# Define directories for dataset
DATASET_DIR = "path_to_dataset"  # Replace with the actual path to your dataset
IMAGE_SIZE = (416, 416)  # Standard size for object detection models like YOLO



In [ ]:
# Function to load and preprocess images
def load_and_preprocess_data(dataset_dir, image_size):
    images = []
    labels = []

In [ ]:
# Assume labels are stored as separate annotation files
 for filename in os.listdir(dataset_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Load image
            img_path = os.path.join(dataset_dir, filename)
            image = cv2.imread(img_path)

            # Resize image
            image_resized = cv2.resize(image, image_size)
            images.append(image_resized)

            # Load corresponding label (this is a placeholder, adapt as needed)
            label_path = img_path.replace(".jpg", ".txt").replace(".png", ".txt")
            if os.path.exists(label_path):
                with open(label_path, 'r') as label_file:
                    labels.append(label_file.read())
            else:
                labels.append("")  # Empty label if not available

    return np.array(images), labels



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

In [ ]:
# Load data
images, labels = load_and_preprocess_data(DATASET_DIR, IMAGE_SIZE)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# Visualize a sample image
plt.imshow(cv2.cvtColor(X_train[0], cv2.COLOR_BGR2RGB))
plt.title("Sample Image")
plt.axis("off")
plt.show()


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# Step 2: Model Training
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Define a simple CNN model for object detection
def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification example
    ])
    return model

# Compile the model
input_shape = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
model = create_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Prepare data for training
X_train_normalized = X_train / 255.0  # Normalize images
X_test_normalized = X_test / 255.0

# Placeholder labels for demonstration (replace with actual labels)
y_train_binary = np.array([1 if "object" in label else 0 for label in y_train])
y_test_binary = np.array([1 if "object" in label else 0 for label in y_test])

# Train the model
history = model.fit(X_train_normalized, y_train_binary, validation_data=(X_test_normalized, y_test_binary), epochs=10, batch_size=32)

# Save the model
model.save("object_detection_model.h5")

# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'X_train' is not defined

In [ ]:
# Step 3: Model Testing and Evaluation
# Load the trained model
from tensorflow.keras.models import load_model

model = load_model("object_detection_model.h5")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test_binary)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predict on a sample test image
sample_image = X_test_normalized[0:1]  # Select the first image from the test set
prediction = model.predict(sample_image)

# Display the result
plt.imshow(cv2.cvtColor(X_test[0], cv2.COLOR_BGR2RGB))
plt.title(f"Predicted: {'Object' if prediction[0] > 0.5 else 'No Object'}")
plt.axis("off")
plt.show()

In [ ]:
# Step 4: Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply data augmentation
augmented_data = data_gen.flow(X_train_normalized, y_train_binary, batch_size=32)

# Retrain the model with augmented data
history_augmented = model.fit(augmented_data, validation_data=(X_test_normalized, y_test_binary), epochs=10, batch_size=32)

# Plot augmented training history
plt.plot(history_augmented.history['accuracy'], label='Augmented Training Accuracy')
plt.plot(history_augmented.history['val_accuracy'], label='Validation Accuracy')
plt.title('Augmented Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Step 5: IoT Integration (Example)
import requests

def send_prediction_to_iot(prediction, endpoint_url="http://iot-server.example.com/api/predictions"):
    payload = {"prediction": float(prediction[0][0])}
    response = requests.post(endpoint_url, json=payload)
    print(f"IoT Server Response: {response.status_code}, {response.text}")

# Example prediction and sending to IoT
iot_prediction = model.predict(sample_image)
send_prediction_to_iot(iot_prediction)
